<a href="https://colab.research.google.com/github/jimjoseph51000/fpl-simulator/blob/main/Player_Swapout_jj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import requests, json
from pprint import pprint

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [3]:
% cd /gdrive/MyDrive/CSE_519_DSF/Project\ -\ FPL

/gdrive/MyDrive/CSE_519_DSF/Project - FPL


In [4]:
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt

from simulator import *
from model import *
from scout import *
# from util import *

In [5]:
# main code
fpl_manager_id = '2757'
current_week = 10
env = FPLSimulator(current_week, fpl_manager_id, req_cols=['stats.ict_index','element_type'])

# plot_pts = env.sample_visualization(10000)
# print(plot_pts)

(620,) (620, 10) (620, 10) (2, 620, 10)
cumsum of per_week_total_points:  [ 80. 166. 238. 297. 355. 401. 436. 484. 540. 588.]
(15, 10) (15, 10) (15, 10) (2, 15, 10)


In [9]:
min_balance = 50
current_balance = 10
K = 10
for week_idx in range(10):
  # week_idx = 0
  

  scout = Scout(env, week_idx, min_balance)
  out_ids = scout.find_transfer_out_candidates(K)
  in_ids = np.array([1,3,4,5,6,101,200,330,500,600])
  _ , current_balance = scout.get_transfer_in_out_players(current_balance, in_ids, out_ids)
  print(week_idx, current_balance)

0 60.0
1 55.0
2 61.0
3 99.0
4 94.0
5 99.0
6 81.0
7 97.0
8 109.0
9 98.0


In [8]:
env.actual_player_other_data[1][:,week_idx]

array(['GKP', 'DEF', 'DEF', 'DEF', 'MID', 'MID', 'MID', 'MID', 'FWD',
       'FWD', 'FWD', 'GKP', 'DEF', 'DEF', 'MID'], dtype=object)

In [9]:
env.actual_players_ids[:,week_idx]

array([ 30, 275, 418, 237, 233, 215, 289, 485, 413, 337, 177, 376, 245,
       208, 145])

In [10]:
np.isin(env.actual_players_ids[:,week_idx],out_ids)

array([ True,  True,  True, False, False, False, False,  True, False,
       False, False,  True, False, False, False])

In [11]:
env.actual_player_other_data[1][:,week_idx][np.isin(env.actual_players_ids[:,week_idx],out_ids)]

array(['GKP', 'DEF', 'DEF', 'MID', 'GKP'], dtype=object)

In [12]:
in_ids = np.array([1,3,4,5,6])
env.all_player_other_data[1][:,week_idx][in_ids - 1]

array(['GKP', 'MID', 'FWD', 'DEF', 'FWD'], dtype=object)

In [13]:
scout.get_transfer_in_out_players(current_balance, in_ids, out_ids)

[3] [485]
[[ 20.  35.  70.  15.  55.]
 [-35. -20.  15. -40.   0.]
 [  5.  20.  55.   0.  40.]
 [ 10.  25.  60.   5.  45.]
 [  5.  20.  55.   0.  40.]]


(array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [3, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 20.0)

In [ ]:
assert(0)

In [ ]:
env.all_player_other_data_cols.index("element_type")

In [ ]:
test = np.zeros_like(env.actual_players_ids)
test[env.actual_players_ids[:,0] == 233, 0] = 1

In [ ]:
from 

In [ ]:
env.actual_player_other_data.shape , env.all_player_other_data.shape, env.all_player_other_data_cols

In [ ]:
def get_sortd_desc_players(fpl_manager_id,week_no):
  r = requests.get('https://fantasy.premierleague.com/api/entry/'+str(fpl_manager_id)+'/event/'+'1'+'/picks/').json()
  fpl_team = pd.json_normalize(r['picks'])
  players_id_list = fpl_team['element'].tolist()
  players_df_cost = players_df.loc[players_df['id'].isin(players_id_list), ['id','now_cost','element_type']]
  r = requests.get(f"https://fantasy.premierleague.com/api/event/{week_no}/live/").json()
  gw_player_df = pd.json_normalize(r['elements'])
  merged_player_df = pd.merge(gw_player_df, players_df_cost, on=["id"])
  merged_player_df['ROI'] = merged_player_df['stats.total_points']/merged_player_df['now_cost']
  sorted_team_players = merged_player_df.sort_values(['ROI','stats.ict_index'], ascending=[False, False])
  return sorted_team_players

In [ ]:
# make sure you this convertion to float since the original variable is in object type
env.actual_player_other_data[0].astype('float')


In [ ]:
env.actual_player_other_data

In [ ]:
# def rankings_table():
roi = env.actual_players_points / env.actual_player_cost # (15,10)
ict_index = env.actual_player_other_data[0].astype('float') # (15,10)
# players_ranking_info = np.concatenate((roi[:,:,np.newaxis], ict_index[:,:,np.newaxis]), axis=2) # (15,10,2)


In [ ]:
# players_ranking_info[:,0,:].shape , players_ranking_info[:,0,:]

In [ ]:
# our current list of players
env.actual_players_ids

In [ ]:
# sorted by roi
roi_sort_idx = np.argsort(roi,axis=0)
dates_idx = np.broadcast_to(np.arange(current_week), roi_sort_idx.shape)
env.actual_players_ids[roi_sort_idx, dates_idx]

In [ ]:
all_player_cost = pd.read_csv("Player_Cost_Weekwise/all_player_costs.csv")
# df_type = df_type.set_index("id")
# return df_type


In [ ]:
env.all_player_cost

In [ ]:
# all_player_cost.set_index([all_player_cost.index.astype(np.int)])
all_player_cost.index = all_player_cost.index.map(int)


In [ ]:
env.all_player_ids[:10]

In [ ]:
all_player_cost.shape

In [ ]:
env.all_player_ids